# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\List of Store Opening20240424.xlsx", sheet_name = 'PowerBI')
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,FC Home Center Robinsons Sta Rosa,Robinsons Santa Rosa,For Opening,2024-04-24,MB6UOCA,South Luzon 2,Kenneth Mendoza,FC Home


In [4]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [5]:
df['BrandStores'].value_counts()

BrandStores
Junliz    1
Name: count, dtype: int64

In [6]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MPBE087    1
Name: count, dtype: int64

In [7]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,Junliz,Junliz Expo,Standalone,POS Operational,"May 1, 2024",MPBE087,NCR North,Sander Maghirang


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      1 non-null      object
 1   POS NAME         1 non-null      object
 2   STORE LOCATION   1 non-null      object
 3   STATUS           1 non-null      object
 4   DATE ACTIVATION  1 non-null      object
 5   PURPLE KEY       1 non-null      object
 6   REGION           1 non-null      object
 7   TSM              1 non-null      object
dtypes: object(8)
memory usage: 192.0+ bytes


In [9]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [10]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "May 1, 2024" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
print(df['DateOpened'].dtype)
# datetime64[ns]

datetime64[ns]


In [11]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-04-24 00:00:00


In [12]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    1 non-null      object
 1   POS            1 non-null      object
 2   StoreLocation  1 non-null      object
 3   Status         1 non-null      object
 4   DateOpened     1 non-null      object
 5   PurpleKey      1 non-null      object
 6   Region         1 non-null      object
 7   TSM            1 non-null      object
dtypes: object(8)
memory usage: 192.0+ bytes


In [13]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    1 non-null      object        
 1   POS            1 non-null      object        
 2   StoreLocation  1 non-null      object        
 3   Status         1 non-null      object        
 4   DateOpened     1 non-null      datetime64[ns]
 5   PurpleKey      1 non-null      object        
 6   Region         1 non-null      object        
 7   TSM            1 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 192.0+ bytes


In [15]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Robinsons Sta Rosa,Robinsons Santa Rosa,For Opening,2024-04-24,MB6UOCA,South Luzon 2,Kenneth Mendoza


In [16]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [17]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e0136430-d2a0-4188-bbbf-9d5452ab9ac3>

In [18]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID f552a5bd-e006-43a9-81b4-5fd36fa08b6a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(559, 8)


In [19]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 2fa29059-9d19-4cca-9d08-ea98e8bc97a0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [20]:
dfsil.BrandStores.value_counts()

BrandStores
Rulls               124
FC Home             108
Home Along           90
ESLA                 51
MemoXpress           37
Willy and Sons       26
Xtreme               22
Ansons               15
Homeworks            13
Automatic Centre     11
Kservico             10
Faith Glorious        9
Jhunpyo               8
Workstation           6
Power Mac Center      5
A2RJ                  3
Sogo Home             3
Willy & Sons          3
Affordables           2
KUM Home Center       2
PHPS                  2
Junliz                2
Home Factory          2
Hardware Sugar        2
Dizon Furniture       1
JTG                   1
KUM Home              1
Name: count, dtype: int64

In [21]:
dfsil.shape

(559, 8)

In [22]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 560 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [23]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID 5ac12a61-8e71-46e5-bdc6-9ea428f01dbf successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (560, 8)


In [24]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
550,FC Home,FC Home Center San Pablo,Standalone,POS Operational,2023-09-14,MADP0JZ,South Luzon 2,Christian Klide Alberto
551,FC Home,FC Home Center Sta Cruz,Standalone,POS Operational,2023-09-14,M1ZYHKU,South Luzon 2,Christian Klide Alberto
552,FC Home,FC Home Center Cabuyao 1,Standalone,POS Operational,2023-09-14,M5D8JOH,South Luzon 2,Christian Klide Alberto
553,FC Home,FC Home Center Binan,Central Mall Laguna,POS Operational,2023-09-14,MYRK3EJ,South Luzon 2,Christian Klide Alberto
554,FC Home,FC Home Center Mamatid,Standalone,POS Operational,2023-09-14,MC31AV7,South Luzon 2,Christian Klide Alberto
555,FC Home,FC Home Center Siniloan,Standalone,POS Operational,2023-09-15,MANB6XQ,South Luzon 2,Christian Klide Alberto
556,Workstation,Workstation Home Appliances Pavilion Mall 1,Pavillon Mall,POS Operational,2023-10-13,MN8MWPJ,South Luzon 2,Christian Klide Alberto
557,Workstation,Workstation Home Appliances Target Mall,Target Mall,POS Operational,2023-10-13,M8I4QGT,South Luzon 2,Christian Klide Alberto
558,Workstation,Workstation Home Appliances Central Mall,Central Mall,POS Operational,2023-10-13,MTNCOR5,South Luzon 2,Christian Klide Alberto
559,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto


In [25]:
df['BrandStores'].value_counts()

BrandStores
Rulls               124
FC Home             109
Home Along           90
ESLA                 51
MemoXpress           37
Willy and Sons       26
Xtreme               22
Ansons               15
Homeworks            13
Automatic Centre     11
Kservico             10
Faith Glorious        9
Jhunpyo               8
Workstation           6
Power Mac Center      5
A2RJ                  3
Sogo Home             3
Willy & Sons          3
Affordables           2
KUM Home Center       2
PHPS                  2
Junliz                2
Home Factory          2
Hardware Sugar        2
Dizon Furniture       1
JTG                   1
KUM Home              1
Name: count, dtype: int64

In [26]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,3,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,4,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
12,522,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
13,524,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
14,529,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
15,530,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
4,444,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
5,445,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
6,456,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo
8,458,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo


In [27]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [28]:
df[df['PurpleKey']== 'MB6UOCA']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Robinsons Sta Rosa,Robinsons Santa Rosa,For Opening,2024-04-24,MB6UOCA,South Luzon 2,Kenneth Mendoza


In [29]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c9fbae4b-c331-4185-a6ec-eea10848cb51>

In [30]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=da1fb8d6-988e-4e7e-a2b3-96a57e2b6847>

In [31]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=66b365b3-3dcf-42e9-9f8f-7bb1ec4834e5>

In [32]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID 4065a5ab-4a4f-4793-b861-1f2234037aa3 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(560, 8)

In [34]:
df[df['BrandStores']=='FC Home'].sort_values(by='DateOpened', ascending=TRUE)

Query is running:   0%|          |


NameError: name 'TRUE' is not defined

In [39]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
2,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
3,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
4,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
5,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
6,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
7,Rulls,Mobilecraze K1 PG Marikina,Puregold Marikina,For Opening,2024-04-15,M9KLTD1,NCR East,Albee David
8,Ansons,Ansons Ortigas,Standalone,For Opening,2024-04-15,MAQ1TK4,NCR East,Albee David
9,MemoXpress,Memoxpress Sta Lucia East Grand Mall,Sta Lucia Mall,For Opening,2024-04-15,MRTAIQG,NCR East,Albee David


In [40]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
325,ESLA,ESLA Balagtas,Standalone,For Opening,2024-04-22,MAS4LQ3,North Luzon 2,Rachelle Belmonte
251,Willy and Sons,Willy And Sons Ligao,Standalone,For Opening,2024-04-22,MISRA7K,South Luzon 3,Jan jan Dela Cruz
245,ESLA,ESLA Tanauan 2,Standalone,For Opening,2024-04-22,MCLWZDG,South Luzon 3,Jan jan Dela Cruz
246,Willy and Sons,Willy And Sons Daet,Standalone,For Opening,2024-04-22,MVIDTNQ,South Luzon 3,Jan jan Dela Cruz
247,Willy and Sons,Willy And Sons Daraga,Standalone,For Opening,2024-04-22,MDLENQF,South Luzon 3,Jan jan Dela Cruz
248,Willy and Sons,Willy And Sons Bulan,Standalone,For Opening,2024-04-22,MHQ2U64,South Luzon 3,Jan jan Dela Cruz
249,Willy and Sons,Willy And Sons Gubat,Standalone,For Opening,2024-04-22,MGZ2JX0,South Luzon 3,Jan jan Dela Cruz
250,Willy and Sons,Willy And Sons Ragay,Standalone,For Opening,2024-04-22,MBK1H6T,South Luzon 3,Jan jan Dela Cruz
252,Willy and Sons,Willy And Sons Gumaca,Standalone,For Opening,2024-04-22,M60QVX9,South Luzon 3,Jan jan Dela Cruz
243,Willy and Sons,Willy And Sons Pili,Standalone,For Opening,2024-04-22,M0IRHDQ,South Luzon 3,Jan jan Dela Cruz


In [41]:
df[df['PurpleKey'] =='M3Z10RP']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
422,Ansons,Ansons Greenhills,Greenhills,For Opening,2024-04-15,M3Z10RP,NCR South 1,Klein Justine Tamayo


In [42]:
df[df['BrandStores']=='ESLA']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
30,ESLA,ESLA Malabon,Standalone,For Opening,2024-04-22,M56ROOZ,NCR East,Albee David
31,ESLA,ESLA Balanga 2,Standalone,For Opening,2024-04-22,MLANGU7,NCR East,Albee David
49,ESLA,ESLA San Mateo 1,Standalone,For Opening,2024-02-15,MD6S28C,NCR East,Charm Moltio
50,ESLA,ESLA Pasig 2,Standalone,For Opening,2024-02-15,MRX4NDS,NCR East,Charm Moltio
51,ESLA,ESLA Morong,Standalone,For Opening,2024-02-15,MV86NSM,NCR East,Charm Moltio
52,ESLA,ESLA Taytay,Standalone,For Opening,2024-02-15,MWI9HXN,NCR East,Charm Moltio
53,ESLA,ESLA Pasig 3,Standalone,For Opening,2024-02-15,MBS5T2W,NCR East,Charm Moltio
54,ESLA,ESLA Marikina 2,Standalone,For Opening,2024-02-15,MTGEZ1S,NCR East,Charm Moltio
55,ESLA,ESLA Antipolo,Standalone,For Opening,2024-02-15,MDT8FNQ,NCR East,Charm Moltio
56,ESLA,ESLA Cogeo,Standalone,For Opening,2024-02-15,M9N5R2P,NCR East,Charm Moltio
